In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_astradb import AstraDBVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
import PyPDF2 as pdf
import os

c:\a\envs\resume\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash",temperature = 0)

print(llm.invoke("What is the capital of India").content)

The capital of India is **New Delhi**. 



In [3]:
def pdf_to_text(path):
    reader=pdf.PdfReader(path)
    text=""
    for page in range(len(reader.pages)):
        page=reader.pages[page]
        text+=str(page.extract_text())
    return text

In [4]:
text = pdf_to_text(r"C:\Projects\Resume_Shortlister\data\My_Resume.pdf")

In [6]:
description = """Machine Learning(ML),Deep Learning(DL),Generative AI,RAG"""

In [12]:
prompt2 = f"""You are a a skilled or very experience ATS(Application Tracking System)
with a deep understanding of tech field,software engineering,data science ,data analyst
and big data engineer. Your task is to read the resume and rate it in terms of percentage of keywords 
matching in the gievn job decription. Description{description} .Resume{text}.answer in terms of percentage only"""

In [13]:
llm.invoke(prompt2).content

'75% \n'